ONNX is an open format to represent DL models that allows models to be reused across frameworks.

In [7]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
df = pd.read_csv('data/wine_data.csv')
features = df.drop('Class', axis=1)
labels = df[['Class']]

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(X_test.values).float()
ytrain_ = torch.from_numpy(y_train.values).long().view(1, -1)[0]
ytest_ = torch.from_numpy(y_test.values).long().view(1, -1)[0]

input_size = 13
output_size = 3
hidden_size = 100

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    

    def forward(self, X):
        X = torch.sigmoid(self.fc1(X))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [10]:
model = Net()
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()
EPOCHS = 1000
for epoch in range(EPOCHS):
    optimizer.zero_grad()
    y_pred = model(Xtrain_)
    loss = loss_fn(y_pred, ytrain_)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss.item():.4f}')

Epoch: 100, Loss: 0.0965
Epoch: 200, Loss: 0.0361
Epoch: 300, Loss: 0.0434
Epoch: 400, Loss: 0.0307
Epoch: 500, Loss: 0.0219
Epoch: 600, Loss: 0.0113
Epoch: 700, Loss: 0.0055
Epoch: 800, Loss: 0.0031
Epoch: 900, Loss: 0.0020
Epoch: 1000, Loss: 0.0014


In [11]:
# A sample dummy input which specifies the input shape is also passed along with the model
sample = np.array(X_test)
sample_tensor = torch.from_numpy(sample).float()

In [12]:
torch.onnx.export(model, sample_tensor, 'models/classifier.onnx', export_params=True, verbose=True)


graph(%input.1 : Float(36, 13, strides=[1, 36], requires_grad=0, device=cpu),
      %fc1.weight : Float(100, 13, strides=[13, 1], requires_grad=1, device=cpu),
      %fc1.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %fc2.weight : Float(100, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %fc2.bias : Float(100, strides=[1], requires_grad=1, device=cpu),
      %fc3.weight : Float(3, 100, strides=[100, 1], requires_grad=1, device=cpu),
      %fc3.bias : Float(3, strides=[1], requires_grad=1, device=cpu)):
  %7 : Float(36, 100, strides=[100, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1](%input.1, %fc1.weight, %fc1.bias) # C:\Users\Safiuddin\anaconda3\lib\site-packages\torch\nn\functional.py:1848:0
  %8 : Float(36, 100, strides=[100, 1], requires_grad=1, device=cpu) = onnx::Sigmoid(%7) # <ipython-input-9-c4af461c3af3>:23:0
  %9 : Float(36, 100, strides=[100, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=

In [ ]:
# Importing to Caffe2
import onnx
import caffe2.python.onnx.backend as c2
new_model = onnx.load('models/classifier.onnx')
prepared_backend = c2.prepare(new_model)
